# Base

## Imports

In [1]:
import numpy as np
import torch
import random

from torchsummary import summary
import torch.nn as nn
import jsonlines
import yaml

import os

from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from torchvision.utils import make_grid
from torchviz import make_dot

## Seed

In [2]:
def seed_all(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic=True
    torch.backends.cudnn.benchmark= False
    random.seed(seed)

In [3]:
###################################################################

In [4]:
seed_all()

## Mapper

In [5]:
class ConfigMapper:
    """Class for creating ConfigMapper objects.

    This class can be used to create custom configuration names using YAML files.
    For each class or object instantiated in any modules,
    the ConfigMapper object can be used either with the functions,
    or as a decorator to store the mapping in the function.

    Attributes
    ----------

    Methods
    -------
    
    """
    dicts = {
        "models":{},
        "trainers":{},
        "metrics":{},
        "losses":{},
        "optimizers":{},
        "schedulers":{},
        "devices":{},
        "transforms":{},
        "params":{}
    }
    @classmethod
    def map(cls,key,name):
        """
        Map a particular name to an object, in the specified key

        Parameters
        ----------
            name : str
                The name of the object which will be used.
            key : str
                The key of the mapper to be used.
        """
        def wrap(obj):
            if(key in cls.dicts.keys()):
                cls.dicts[key][name]=obj
            else:
                cls.dicts[key] = {}
                cls.dicts[key][name]=obj
            return obj
        return wrap

    @classmethod
    def get_object(cls,key,name):
        """
        """
        try:
            return cls.dicts[key][name]
        except:
            raise NotImplementedError('Key Undefined.')
configmapper = ConfigMapper()

## Config

In [6]:
def load_yaml(path):
    """
    Function to load a yaml file and
    return the collected dict(s)

    Parameters
    ----------
    path : str
        The path to the yaml config file

    Returns
    -------
    result : dict
        The dictionary from the config file
    """

    assert isinstance(path,str), "Provided path is not a string"
    try:
        f = open(path,'r')
        result = yaml.load(f,Loader=yaml.Loader)
    except FileNotFoundError as e:
        # Adding this for future functionality
        raise e
    return result
class Config:
    """Config Class to be used with YAML configuration files

    This class can be used to address keys as attributes.
    Ensure that there are no spaces between the keys.
    Only objects of type dict can be converted to config.

    Attributes
    ----------
    _config : dict,
        The dictionary which is formed from the
        yaml file or custom dictionary

    Methods
    -------
    as_dict(),
        Return the config object as dictionary

        Possible update:
        ## Can be converted using __getattr__ to use **kwargs
        ## with the Config object directly.

    set_value(attr,value)
        Set the value of a particular attribute.
    """
    def __init__(self,*,path=None,dic=None):
        """
        Initializer for the Config class

        Needs either path or the dict object to create the config

        Parameters
        ----------
        path: str, optional
            The path to the config YAML file.
            Default value is None.
        dic : dict, optional
            The dictionary containing the configuration.
            Default value is None.
        """
        if(path):
            self._config = load_yaml(path)
        elif(dict):
            self._config = dic
        else:
            raise Exception('Need either path or dict object to instantiate object.')
        # self.keys = self._config.keys()

    def __getattr__(self,attr):
        """
        Get method for Config class. Helps get keys as attributes.

        Parameters
        ----------
        attr: The attribute name passed as <object>.attr

        Returns
        -------
        self._config[attr]: object or Config object
            The value of the given key if it exists.
            If the value is a dict object,
            a Config object of that dict is returned.
            Otherwise, the exact value is returned.

        Raises
        ------
        KeyError() if the given key is not defined.
        """
        if(attr in self._config):
            if(isinstance(self._config[attr],dict)):
                return Config(dic=self._config[attr])
            else:
                return self._config[attr]
        else:
            raise KeyError(f"Key:{attr} not defined.")
    def set_value(self,attr,value):
        """
        Set method for Config class. Helps set keys in the _config.

        Parameters
        ----------
        attr: The attribute name passed as <object>.attr
        value: The value to be stored as the attr.
        """
        self._config[attr]=value

    def __str__(self):
        """ Function to print the dictionary
         contained in the object."""
        return self._config.__str__()

    def as_dict(self):
        """Function to get the config as dictionary object"""
        return dict(self._config)                

In [7]:
def convert_params_to_dict(params):
    dic = {}
    for k,v in params.as_dict():
        try:
            obj = configmapper.get_object('params',v)
            dic[k]=v
        except:
            print(f"Undefined {v} for the given key: {k} in mapper        ,storing original value")
            dic[k]=v
        return value

In [8]:
######################################################################

In [9]:
main_config = Config(path = '../configs/default.yaml')
model_config = Config(path = '../configs/models/unimodal/image.yaml')
trainer_config = Config(path = '../configs/trainer.yaml')
data_config = Config(path='../configs/data.yaml')

## Reader

In [10]:
class JsonlReader:
    """A class for reading jsonl files easily

    Attributes
    ----------
    _path : str
        The file location of the jsonl file
    _reader: jsonlines.Reader object
        The object which reads the files and keeps a tracker on the file
    _size : int
        Number of lines to be read
    _read : int
        Number of lines read so far

    Methods
    -------
    size()
        Getter method for _size.
    path()
        Getter method for _path.
    resetReader()
        Resets the _reader, starts over again.
    readNext(loop=False)
        Read the next line from the file. Loop over if loop=True.
    read(count=None, loop=False)
        Read the next count lines from the file, until self._size lines are read in total. Loop over if loop=True.
    close()
        Closes the reader of the file.
    """

    def __init__(self,path,size=None):
        """
        Initializes the JsonReader object.

        Parameters
        ----------
        path : string
            The path to the .jsonl file.
        size : int, optional
            The number of lines to be read from the file.
            If None, size is set to total number of lines.
            (default is None).
        """

        self._path = path
        self._reader = jsonlines.open(path,'r')
        if(size):
            self._size = size
        else:
            self._size = len(list(iter(jsonlines.open(path,'r'))))
        self._read = 0
    @property
    def size(self):
        """
        Function to get the _size attribute.

        Returns
        -------
            self._size : int
                The number of lines which will be read from the file
        """
        return self._size

    @property
    def path(self):
        """
        Function to get the _path attribute.
                Returns
        -------
            self._path : str
                The path of the the file
        """

        return self._path

    def resetReader(self):
        """Function to reset the reader attribute

        This function starts reading the file from the beginning.
        """

        self._reader = jsonlines.open(self._path,'r')
        self._read = 0
    def readNext(self,loop=False):
        """
        Function to read the next line from the path.

        If the loop parameter is set to True,
        then the reader starts over if the file ends.

        Raises an Exception if the file ends,
        and if the loop parameter is False.


        Parameters
        ----------
            loop : bool, optional
               Signifies whether the reader should start reading again
               if the end of file is reached. (default is False)
        Returns
        -------
            result: dict
                Returns next line of the file as a dict.
        """
        if(self._read>=self._size):
            print("Max count reached. Reset to start again.")
            return
        try:
            result = self._reader.read()
            self._read+=1
        except EOFError as e:
            if(loop==False):
                print('End of File reached.')
                return
            else:
                self.resetReader()
                try:
                    result =  self._reader.read()
                    self._read+=1
                except:
                    print('Empty File. Aborting Read')
                    return
        return result
    def read(self,count=None,loop=False):
        """
        Function to read the next count lines from the file.

        If the loop parameter is set to True,
        then the reader starts over if the file ends.

        Raises an Exception if the file ends,
        and returns the collected lines.


        Parameters
        ----------
            count : int, optional
                The number of lines to be read. If None,
                reads all the remaining lines. (default is None)

            loop : bool, optional
               Signifies whether the reader should start reading again
               if the end of file is reached. (default is False).
               Insignificant if the count is None.
        Returns
        -------
            lines: list of dict
                Returns the lines collected from the file.
        """

        lines = []
        if(count is None):
            for line in self._reader:
                self._read+=1
                if(self._read<=self._size):
                    lines.append(line)
                else:
                    break
            return lines
        for line in range(count):
            try:
                result = self.readNext(loop=loop)
                if(result):
                    lines.append(result)
                else:
                    raise Exception('None received at self.readNext')
            except:
                print('Error occurred. Returning collected lines so far.')
        return lines
    def close(self):
        """ Function to close the reader """
        self._reader.close()

## Logger

In [11]:
class Logger():
    """

    """
    def __init__(self,model,trainer,log_dir,comment=None):
        """ Initializer for Logger Class
        #Arguments:

        """
        self.model_path = os.path.join(log_dir,model,trainer)
        self.writer = SummaryWriter(log_dir=self.model_path,comment=comment)
        try:
            if(not os.exists(log_dir)):
                os.makedir(log_dir)
            if(not(os.exists(self.model_path))):
                os.makedir(self.model_path)
            else:
                print("Directory Already Exists.")
        except:
            print("Failed to Create Directory.")


    def save_params(self,param_list,param_name_list,epoch,batch_size,batch=None,combine=False,combine_name=None):
        if(combine==False):
            for i in range(len(param_list)):
                if(isinstance(param_list[i],Variable)):
                    param_list[i] = param_list[i].data.cpu().numpy()
                    self.writer.add_scalar(param_name_list[i],param_list[i],Logger._global_step(epoch,batch_size,batch))

        else:
            scalar_dict = dict(zip(param_name_list,param_list))
            self.writer.add_scalars(combine_name,scalar_dict,Logger._global_step(epoch,batch_size,batch))


    def save_batch_images(self,image_name,image_batch,epoch,batch_size,batch=None,dataformats = 'CHW'):
        self.writer.add_images(image_name,image_batch,Logger._global_step(epoch,batch_size,batch),dataformats=dataformats)

    def save_prcurve(self,labels,preds,epoch,batch_size,batch=None):
        self.writer.add_pr_curve('pr_curve',labels,preds,Logger._global_step(epoch,batch_size,batch))

    def save_hyperparams(self,hparam_list,hparam_name_list,metric_list,metric_name_list):
        self.writer.add_hparams(zip(metric_name_list,metric_list),zip(hparam_name_list,hparam_list))

    def save_models(self,model_list,model_names_list,epoch):
        for model_name,model in zip(model_names_list,model_list):
            torch.save(model.state_dict(),os.path.join(self.model_path,model_name))

    def save_fig(self,fig,fig_name,epoch,batch_size,batch=None):
        self.writer.add_figure(fig_name,fig,Logger._global_step(epoch,batch_size,batch))

    def display_params(params_list,params_name_list,epoch,num_epochs,batch_size,batch):
        for i in range(len(params_list)):
            if isinstance(params_list[i],Variable):
                params_list[i] = params_list[i].data.cpu().numpy()
        print('Epoch: {}/{}, Batch: {}/{}'.format(epoch,num_epochs,batch,batch_size))
        for i in range(len(params_list)):
            print('{}:{}'.format(params_name_list[i],params_list[i]))

    def draw_model_architecture(model,output,input,input_name,save_name):
        make_dot(output,params = dict(list(model.named_parameters()))+[(input_name,input)])

    def __del__(self):
        self.writer.close()

    @staticmethod
    def _global_step(epoch,batch_size,batch):
        if(batch):
            return epoch*batch_size + batch
        else:
            return epoch

## Check Point

## Visualize

# Data

## Processor

In [12]:
def map_dict_to_obj(dic):
    result_dic = {}
    if(dic is not None):
        for k,v in dic.items():
            if(isinstance(v,dict)):
                result_dic[k]=map_dict_to_obj(v)
            else:
                try:
                    obj = configmapper.get_object('params',v)
                    result_dic[k]=obj
                except:
                    result_dic[k]=v
    return result_dic

from torchvision import transforms

def get_image_processor(processor):
    transformations = []
    if(processor.type=='torchvision'):
        for param in processor.params:
            transformations.append(configmapper.get_object('transforms',param['type'])(**map_dict_to_obj(param['params'])))
    return transforms.Compose(transformations)

In [13]:
######################################################

## Dataset

In [14]:
from torch.utils.data import Dataset
class MemesDataset(Dataset):
  """ Dataset class for Hateful Memes """
  def __init__(self,config,typ='train'):
    """Init Function for the Dataset Class

    Parameters:
    ------
    config : The Config object containing configuration for data
    typ : String value specifying whether it is train,dev or test
    
    """

    self._config = config
    self.type = typ
    self.reader = JsonlReader(self._config.annotations.as_dict()[typ])
    self.annotations = self.reader.read()
    self.transform = get_image_processor(self._config.image_processor)
    
  def __len__(self):
    """Function to return the size of the data"""
    return self.reader.size
  def __getitem__(self,idx):
    record_dic=self.annotations[idx]
    img = Image.open(os.path.join(self._config.data_dir,record_dic['img'])).convert('RGB')
    text = record_dic['text']
    if self.transform:
        img = self.transform(img)
    label =record_dic['label']
    if(self.type in ['train','dev']):
      if(self._config.get_image):
        if(self._config.get_text):
          return img,text,label
        else:
          return img,label
      elif(self._config.get_text):
        return text,label
      else:
        raise Exception('Need atleast some features to return.')
        return
    else:
      if(self._config.get_image):
        if(self._config.get_text):
          return img,text
        else:
          return img
      elif(self._config.get_text):
        return text
      else:
        raise Exception('Need atleast some features')
        return

In [15]:
#######################################

In [16]:
# for i,batch in enumerate(DataLoader(dataset,batch_size=12)):
#     print(batch[0],batch[1])
#     if(i>2):
#         break

# Model

In [17]:
@configmapper.map("models","unimodal")
class Unimodal(nn.Module):
  def __init__(self,config):
    super(Unimodal,self).__init__()
    self._config = config
    self.mode = config.mode
    if(self.mode == 'image'):
        self.modal_encoder,in_features = get_backbone(config.modal_encoder)
        self.classifier = get_classifier(config.classifier)
        self.flatten = nn.Flatten()

  def forward(self,x):
    if(self.mode=='image'):
        x = self.flatten(self.modal_encoder(x))
        x = self.classifier(x)
    return x

# Modules

In [18]:
def get_backbone(modal_encoder):
    if(modal_encoder.type=='resnet152'):
        model = torch.hub.load('pytorch/vision:v0.6.0','resnet152',pretrained=modal_encoder.params.pretrained)
    in_features = model.fc.in_features
    if(modal_encoder.params.remove_classifier):
      model = nn.Sequential(*list(model.children())[:-1])
    return model,in_features
def get_classifier(classifier):
    layers =[]
    if(classifier.custom_layers is None):
        if(classifier.type =='mlp'):
            for layer in range(classifier.params.num_layers):
                if(layer==0):
                    layers.append(nn.Linear(classifier.params.in_dim,classifier.params.hidden_dims[0]))
                    layers.append(configmapper.get_object('activations',classifier.params.activation.default.name)(**classifier.params.activation.default.params.as_dict()))
                    #print(layers)
                elif(layer==classifier.params.num_layers-1):
                    layers.append(nn.Linear(classifier.params.hidden_dims[-1],classifier.params.out_dim))
                    layers.append(configmapper.get_object('activations',classifier.params.activation.output.name)(**classifier.params.activation.output.params.as_dict()))
                    #print(layers)
                else:
                    layers.append(nn.Linear(classifier.params.hidden_dims[layer],classifier.params.hidden_dims[layer+1]))
                    layers.append(configmapper.get_object('activations',classifier.params.activation.default.name)(**classifier.params.activation.default.params.as_dict()))
    #print(layers)
    return nn.Sequential(*layers)

In [19]:
from torch.nn import CrossEntropyLoss
configmapper.map('losses','cross_entropy')(CrossEntropyLoss)

torch.nn.modules.loss.CrossEntropyLoss

In [20]:
from sklearn.metrics import roc_auc_score
import torch
from torchnlp.metrics import get_accuracy

@configmapper.map('metrics','binary_auroc')
def binary_auroc(outputs,labels):
    """Function to compute Area Under ROC Curve Score

    Parameters
    ----------
    outputs: torch.Tensor
        Tensor containing the softmax outputs from the model
    labels: torch.Tensor
        Tensor containing the labels (Not one-hot encoded)

    Returns
    -------
    roc_auc_score : int,
        The roc_auc_score computed between the outputs and the labels

    ## Update tips:
    ## More functionality can be added through the parameters.
    ## Custom AUROC function/class can also be defined.
    """
    outputs_index = outputs[:,1]
    return roc_auc_score(labels.detach().numpy(),outputs_index.detach().numpy())

@configmapper.map('metrics','accuracy')
def accuracy(outputs,labels):
    """Function to compute Accuracy Score

    Parameters
    ----------
    outputs: torch.Tensor
        Tensor containing the softmax outputs from the model
    labels: torch.Tensor
        Tensor containing the labels (Not one-hot encoded)

    Returns
    -------
    accuracy_score : int
        The accuracy_score computed between the outputs and the labels

    ## Update tips:
    ## More functionality can be added through the parameters.
    ## Custom function/class can also be defined.
    """
    outputs_argmax = torch.argmax(outputs,dim=1)
    return get_accuracy(labels,outputs_argmax)[0]

In [21]:
from torch.optim import AdamW
configmapper.map('optimizers','adam_w')(AdamW)

torch.optim.adamw.AdamW

In [22]:
import torchvision.transforms as transforms
from PIL import Image
configmapper.map('transforms','Resize')(transforms.Resize)
configmapper.map('transforms','Normalize')(transforms.Normalize)
configmapper.map('transforms','ToTensor')(transforms.ToTensor)
configmapper.map('params','BICUBIC')(Image.BICUBIC)

3

In [23]:
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
configmapper.map('schedulers','cosine_warm')(CosineAnnealingWarmRestarts)

torch.optim.lr_scheduler.CosineAnnealingWarmRestarts

In [24]:
import torch.nn as nn
configmapper.map('activations','relu')(nn.ReLU)
configmapper.map('activations','logsoftmax')(nn.LogSoftmax)

torch.nn.modules.activation.LogSoftmax

# Trainer

## Trainer

In [42]:
import os
from torch.utils.data import Dataset,DataLoader
from tqdm import tqdm

@configmapper.map("trainers","trainer")
class Trainer:
    def __init__(self,config):
        self._config = config
        self.metrics = [configmapper.get_object('metrics',metric) for metric in self._config.main_config.metrics]
        self.train_config = self._config.train
        self.eval_config = self._config.eval
## Train
    def train(self,model,dataset,verbose,tqdm_out=True,eval_dataset=None):

        optim_params = self.train_config.optimizer.params
        if(optim_params):
            optimizer = configmapper.get_object('optimizers',self.train_config.optimizer.type)(model.parameters(),**map_dict_to_obj(optim_params.as_dict()))
        else:
            optimizer = configmapper.get_object('optimizers',self.train_config.optimizer.type)(model.parameters())
        
        scheduler_params = self.train_config.scheduler.params
        if(scheduler_params):
            scheduler = configmapper.get_object('schedulers',self.train_config.scheduler.type)(optimizer,**map_dict_to_obj(scheduler_params.as_dict()))
        else:
            scheduler = configmapper.get_object('schedulers',self.train_config.scheduler.type)(optimizer)
            
        criterion_params = self.train_config.criterion.params
        if(criterion_params):
            criterion = configmapper.get_object('losses',self.train_config.criterion.type)(**map_dict_to_obj(criterion_params.as_dict()))
        else:
            criterion = configmapper.get_object('losses',self.train_config.criterion.type)()
            
        train_loader = DataLoader(dataset,**self.train_config.loader_params.as_dict())
        train_logger = Logger(**self.train_config.log.logger_params.as_dict())
        log_interval = self.train_config.log.log_interval
        eval_interval = self.train_config.eval_interval
        max_steps = self.train_config.max_steps
        log_values = self.train_config.log.values.as_dict()

        step=0
        epoch=0
        break_all=False
        if(tqdm_out):
            pbar = tqdm(total = max_steps)
        print('Starting training...')
        print(max_steps)
        while(step<max_steps):
            epoch+=1
            running_loss = 0
            all_labels = torch.LongTensor()
            all_outputs = torch.Tensor()
            for i,batch in enumerate(train_loader):
                if(step>=max_steps):
                    break_all = True
                    break
                step+=1
                *inputs, labels = [value.to(torch.device(self._config.main_config.device.name)) for value in batch]
                optimizer.zero_grad()
                outputs = model(*inputs)
                loss = criterion(outputs,labels)
                loss.backward()
                all_labels = torch.cat((all_labels,labels),0)
                all_outputs = torch.cat((all_outputs,outputs),0)
                running_loss+=loss.item()
                optimizer.step()
                scheduler.step(epoch + i/len(train_loader))
                
                
                if(step%log_interval==log_interval-1):
                    if(log_values['loss']):
                        train_logger.save_params([loss.item()/self.train_config.loader_params.batch_size],['train_loss'],epoch=epoch,batch_size=self.train_config.loader_params.batch_size,batch=i+1)

                    if(log_values['metrics']):
                        train_logger.save_params([metric(outputs,labels) for metric in self.metrics],[metric for metric in self._config.main_config.metrics],combine=True,combine_name='metrics',epoch=epoch,batch_size=self.train_config.loader_params.batch_size,batch=i+1)

                if(eval_dataset is not None and step%eval_interval==eval_interval-1):
                    self.eval(model,eval_dataset,epoch,i,log_values,criterion)
                pbar.update(1)
                
    
            loss = running_loss/len(train_loader)
            loss_list = [loss]
            loss_name_list = ['train_loss']
            
            if(log_values['loss']):
                train_logger.save_params(loss_list,loss_name_list,epoch=epoch,batch_size=self.train_config.loader_params.batch_size,batch=self.train_config.loader_params.batch_size)

            metric_list =[metric(all_outputs,all_labels) for metric in self.metrics]
            metric_name_list= [metric for metric in self._config.main_config.metrics]
            if(log_values['metrics']):
                train_logger.save_params(metric_list,metric_name_list,combine=True,combine_name='metrics',epoch=epoch,batch_size=self.train_config.loader_params.batch_size,batch=self.train_config.loader_params.batch_size)
            print(f'Train, Epoch:{epoch}')
            print(loss_list+metric_list)
            print(loss_name_list+metric_name_list)

            if(break_all):
                break



        if(tqdm_out):
            pbar.close()

## Evaluate
    def eval(self,model,dataset,epoch,i,log_values,criterion):
        eval_logger = Logger(**self.eval_config.log.logger_params.as_dict())
        eval_loader = DataLoader(dataset,**self.eval_config.loader_params.as_dict())
        all_outputs = torch.Tensor()
        all_labels = torch.LongTensor()
        with torch.no_grad():
            val_loss = 0
            for j,batch in enumerate(eval_loader):
                *inputs, labels = [value.to(torch.device(self._config.main_config.device.name)) for value in batch]
                outputs = model(*inputs)
                loss = criterion(outputs,labels)
                val_loss+=loss.item()
                all_labels = torch.cat((all_labels,labels),0)
                all_outputs = torch.cat((all_outputs,outputs),0)
            val_loss = val_loss/len(eval_loader)
            loss_list = [val_loss]
            loss_name_list = ['eval_loss']
            if(log_values['loss']):
                val_logger.save_params(loss_list,loss_name_list,epoch=epoch,batch_size=self.train_config.loader_params.batch_size,batch=i+1)

            metric_list =[metric(all_outputs,all_labels) for metric in self.metrics]
            metric_name_list= [metric for metric in self._config.main_config.metrics]
            if(log_values['metrics']):
                val_logger.save_params(metric_list,metric_name_list,combine=True,combine_name='metrics',epoch=epoch,batch_size=self.eval_config.loader_params.batch_size,batch=i+1)
            print('Evaluation:')
            print(loss_list+metric_list)
            print(loss_name_list+metric_name_list)


In [43]:
##############

In [44]:
model = Unimodal(model_config)

Using cache found in /home/crocoder/.cache/torch/hub/pytorch_vision_v0.6.0


In [45]:
dataset = MemesDataset(data_config,'train')
dev = MemesDataset(data_config,'dev')

In [46]:
trainer = Trainer(trainer_config)

In [47]:
trainer.train(model,dataset,verbose=True)



  0%|          | 0/2 [00:00<?, ?it/s]

Failed to Create Directory.
Starting training...
2




 50%|█████     | 1/2 [00:17<00:17, 17.94s/it]

100%|██████████| 2/2 [00:31<00:00, 15.71s/it]

Train, Epoch:1
[0.046265465920667555, 0.3473684210526316, 0.5833333333333334]
['train_loss', 'binary_auroc', 'accuracy']


In [35]:
print(trainer_config)

{'name': 'trainer', 'version': 1.0, 'main_config': {'seed': 42, 'metrics': ['binary_auroc', 'accuracy'], 'device': {'name': 'cpu'}}, 'train': {'seed': 42, 'metrics': ['binary_auroc', 'accuracy'], 'device': {'name': 'cpu'}, 'max_steps': 2, 'eval_interval': 1, 'loader_params': {'batch_size': 12, 'num_workers': 4, 'shuffle': True}, 'optimizer': {'type': 'adam_w', 'params': {'lr': 0.01, 'betas': [0.9, 0.998], 'eps': 1e-08}}, 'scheduler': {'type': 'cosine_warm', 'params': {'T_0': 1, 'T_mult': 2, 'eta_min': 1e-07}}, 'criterion': {'type': 'cross_entropy', 'params': None}, 'log': {'log_interval': 1, 'logger_params': {'model': 'unimodal', 'trainer': 'trainer', 'comment': 'Trial for Logger and Trainer', 'log_dir': './logs/train'}, 'values': {'loss': True, 'metrics': True}}}, 'eval': {'seed': 42, 'metrics': ['binary_auroc', 'accuracy'], 'device': {'name': 'cpu'}, 'loader_params': {'batch_size': 12, 'num_workers': 4, 'shuffle': False}, 'log': {'logger_params': {'model': 'unimodal', 'trainer': 'tra